In [192]:
#!cp 1-input.py 2-optimize.py
!chmod +x *.py

In [99]:
#!/usr/bin/env python3

#build_model = __import__('0-sequential').build_model
""" doc """
import tensorflow.keras as K


def build_model(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    for i in range(len(layers)):
        init = K.initializers.VarianceScaling(mode="fan_avg")
        # freg = K.layers.ActivityRegularization(l2=lambtha)
        freg = K.regularizers.l2(lambtha)
        layer = K.layers.Dense(layers[i], input_dim=nx,
                               activation=activations[i],
                               kernel_initializer=init,
                               kernel_regularizer=freg)
        model.add(layer)
        if i != len(layers)-1:
            dropped = K.layers.Dropout(rate=1-keep_prob)
            model.add(dropped)
    return (model)


if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_197 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_104 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_198 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_105 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_199 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'kernel/Regularizer_149/add:0' shape=() dtype=float32>, <tf.Tensor 'kernel/Regularizer_150/add:0' shape=() dtype=float32>, <

In [191]:
def build_model(nx, layers, activations, lambtha, keep_prob):
    inputs = K.Input(shape=(nx,))
    freg = K.regularizers.l2(lambtha)
    x = K.layers.Dense(layers[0], activation=activations[0],
                           kernel_regularizer=freg)(inputs)
    for i in range(1, len(layers)):
        x = K.layers.Dropout(rate=1-keep_prob)(x)
        x = K.layers.Dense(layers[i], activation=activations[i],
                           kernel_regularizer=freg)(x)
        
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return (model)

if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_103 (InputLayer)       (None, 784)               0         
_________________________________________________________________
dense_409 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_309 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_410 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_310 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_411 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf

In [203]:
import tensorflow as tf

build_model = __import__('1-input').build_model
# optimize_model = __import__('2-optimize').optimize_model
def optimize_model(network, alpha, beta1, beta2):
    """ doc """
    optimizer = K.optimizers.Adam(alpha, beta1, beta2)
    network.compile(optimizer,
                    loss="",
                    metrics=None,
                    loss_weights=None,
                    sample_weight_mode=None,
                    weighted_metrics=None,
                    target_tensors=None,
                    distribute=None,
                    )
    return None
if __name__ == '__main__':
    model = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    optimize_model(model, 0.01, 0.99, 0.9)
    print(model.loss)
    print(model.metrics)
    opt = model.optimizer
    print(opt.__class__)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run((opt.lr, opt.beta_1, opt.beta_2))) 

TypeError: ('Expected `target_tensors` to be a list or dict, but got:', 'categorical_crossentropy')